# Week 5 Data Munging
### Chris Richards
### Regis University, Spring 2019  
In this week's exercise I loaded the adult.data data set, created some custom functions, cleaned it by passing those functions to the apply method, and performed a number of grouping and aggregation functions.  


In [319]:
import pandas as pd, numpy as np
from scipy.stats import zscore # used in a later transfrom

In [320]:
df = pd.read_csv('adult.data', header=None)

In [321]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [322]:
df.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K
32560,52,Self-emp-inc,287927,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,>50K


In [323]:
df.shape

(32561, 15)

In [324]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
0     32561 non-null int64
1     32561 non-null object
2     32561 non-null int64
3     32561 non-null object
4     32561 non-null int64
5     32561 non-null object
6     32561 non-null object
7     32561 non-null object
8     32561 non-null object
9     32561 non-null object
10    32561 non-null int64
11    32561 non-null int64
12    32561 non-null int64
13    32561 non-null object
14    32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


According to this, the dataframe has 32561 observations and no missing values (each series has the same number of non-null values).  However, according to the data dictionary missing values were entered as question marks.  
  
  I'll create a copy of the dataframe, dfc, as a working copy.

In [325]:
dfc = df.copy()

I'll add column headers according to the information in the data dictionary.

In [326]:
#rename columns
dfc.columns = ['age', 'workclass', 'final weight', 'education', 
               'education num', 'marital status', 'occupation', 
               'relationship', 'race', 'sex', 'capital gain', 
               'capital loss', 'hours per week', 'native country'
              , 'result']
dfc.head(5)

,age,workclass,final weight,education,education num,marital status,occupation,relationship,race,sex,capital gain,capital loss,hours per week,native country,result
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


#### User Functions
I created several functions to use when cleaning the data.

The count_nulls function takes a vector and returns a count of all the null values. 

In [327]:
def count_nulls(vec):
    null_vec = pd.isnull(vec)
    null_count = np.sum(null_vec)
    return null_count

The replhyphen function takes an object and replaces hyphens with a space.

In [328]:
def replhyphen(data):
    data.replace('-', " ", inplace=True, regex=True)
    return

The data definition document states missing values were entered as question marks.  
I'll use a function with the apply method to count the number of "?" in the file. 

In [329]:
def findquestion(data):
    count = 0
    if data == ' ?':
        count += 1
    return count

The replqmark function will replace the question marks with NaN.  It should be noted that the missing values were entered with a space and a question mark, not just a question mark.  So, " ?", not "?".

In [330]:
def replqmark(data):
    data.replace(' ?',np.NaN , inplace=True)
    return    

#### Call the functions
First, I'll use the apply function with the findquestion function to find the question marks.  In this case, I'm passing in the 'workclass' series.  The result will be the number of question marks in the series - 1,836.

In [331]:
dfc['workclass'].apply(findquestion).sum()

1836

Just to check, I'll use the value_counts method to get a count of the unique values in 'workclass'.  This should show the same number of question marks as my function.

In [300]:
qm = pd.value_counts(df[1]) #workclass
qm

 Private             22696
 Self-emp-not-inc     2541
 Local-gov            2093
 ?                    1836
 State-gov            1298
 Self-emp-inc         1116
 Federal-gov           960
 Without-pay            14
 Never-worked            7
Name: 1, dtype: int64

Next, I'll replace the question marks with the replqmark function passed to apply.

In [301]:
dfc.apply(replqmark)

age               None
workclass         None
final weight      None
education         None
education num     None
marital status    None
occupation        None
relationship      None
race              None
sex               None
capital gain      None
capital loss      None
hours per week    None
native country    None
result            None
dtype: object

At this point, I'd like to check the results of the replacement.  One way is to sum the number of NaNs.  For the 'workclass' series I would expect to see the same number of nulls, 1836, as there were question marks.

In [302]:
dfc.isna().sum()

age                  0
workclass         1836
final weight         0
education            0
education num        0
marital status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital gain         0
capital loss         0
hours per week       0
native country     583
result               0
dtype: int64

Let's check that again with the count_nulls function.

In [303]:
age_null_count = dfc['workclass'].apply(count_nulls)
age_null_count.sum()

1836

I can also doublecheck that result by rerunning the findquestion function on the 'workclass' series.  Since the question marks have been replaced I would expect the result to be 0. That would indicate that they have all been replaced.

In [304]:
dfc['workclass'].apply(findquestion).sum()

0

At this point, I'm ready to replace the hyphens in the data set.  I'll use the apply method with the replhyphen function created earlier.

In [305]:
dfc.apply(replhyphen)
dfc.head()

,age,workclass,final weight,education,education num,marital status,occupation,relationship,race,sex,capital gain,capital loss,hours per week,native country,result
0,39,State gov,77516,Bachelors,13,Never married,Adm clerical,Not in family,White,Male,2174,0,40,United States,<=50K
1,50,Self emp not inc,83311,Bachelors,13,Married civ spouse,Exec managerial,Husband,White,Male,0,0,13,United States,<=50K
2,38,Private,215646,HS grad,9,Divorced,Handlers cleaners,Not in family,White,Male,0,0,40,United States,<=50K
3,53,Private,234721,11th,7,Married civ spouse,Handlers cleaners,Husband,Black,Male,0,0,40,United States,<=50K
4,28,Private,338409,Bachelors,13,Married civ spouse,Prof specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


## Grouping and Aggregation
I'm now ready to run some basic analysis of the dataset.  This will be several instances of using the groupby method to aggregate a series and perform some descriptive statistics on them.

Group the 'education' series and find the mean 'age' of each group.

In [306]:
ed_age = dfc.groupby('education')['age'].mean()
ed_age

education
 10th            37.429796
 11th            32.355745
 12th            32.000000
 1st 4th         46.142857
 5th 6th         42.885886
 7th 8th         48.445820
 9th             41.060311
 Assoc acdm      37.381443
 Assoc voc       38.553546
 Bachelors       38.904949
 Doctorate       47.702179
 HS grad         38.974479
 Masters         44.049913
 Preschool       42.764706
 Prof school     44.746528
 Some college    35.756275
Name: age, dtype: float64

Group 'workclass' and find the mean, minimum and maximum for the 'age' series.

In [307]:
wc_age_hours = dfc.groupby('workclass').age.agg([np.mean,np.min, np.max])
wc_age_hours

,mean,amin,amax
workclass,,,
Federal gov,42.590625,17,90
Local gov,41.751075,17,90
Never worked,20.571429,17,30
Private,36.797585,17,90
Self emp inc,46.017025,17,84
Self emp not inc,44.969697,17,90
State gov,39.436055,17,81
Without pay,47.785714,19,72


Group 'education' and count the number of observations in each education group.  Sort the result in ascending order instead of by the index.

In [334]:
ed = dfc.groupby("education")['education'].count().sort_values()
ed

education
 Preschool          51
 1st-4th           168
 5th-6th           333
 Doctorate         413
 12th              433
 9th               514
 Prof-school       576
 7th-8th           646
 10th              933
 Assoc-acdm       1067
 11th             1175
 Assoc-voc        1382
 Masters          1723
 Bachelors        5355
 Some-college     7291
 HS-grad         10501
Name: education, dtype: int64

Group by 'education' and 'occupation' and get a count of the number of observations in each grouping.  The reset_index method is used to 'flatten' the result.  Flattening this result lists the education factor for each occupational group.

In [309]:
edage_marital = dfc.groupby(['education', 'occupation'])['marital status'].count().reset_index()
pd.DataFrame(edage_marital) # convert the result to a dataframe.

,education,occupation,marital status
0,10th,Adm clerical,38
1,10th,Craft repair,170
2,10th,Exec managerial,24
3,10th,Farming fishing,44
4,10th,Handlers cleaners,71
5,10th,Machine op inspct,101
6,10th,Other service,194
7,10th,Priv house serv,6
8,10th,Prof specialty,9
9,10th,Protective serv,6


Calculate the zscore for the values in the 'age' column grouped by 'sex'.  Use the transform module to apply the zscore function to each value in 'age'.  Note that transform does not reduce the number of values.  

In [310]:
zscore = dfc.groupby('sex')['age'].transform(zscore)
zscore

0       -0.032426
1        0.790291
2       -0.107219
3        1.014669
4       -0.632142
5        0.010117
6        0.866462
7        0.939876
8       -0.418055
9        0.191951
10      -0.182011
11      -0.705559
12      -0.988952
13      -0.555974
14       0.042366
15      -0.406389
16      -1.079521
17      -0.555974
18      -0.107219
19       0.438289
20       0.042366
21       1.223272
22      -0.331596
23       0.266744
24       1.580082
25       1.239047
26      -1.528276
27       1.089461
28      -0.032426
29       0.715499
           ...   
32531   -0.489418
32532   -0.406389
32533    1.089461
32534    0.010117
32535   -1.303899
32536   -0.203969
32537   -0.705559
32538    0.081479
32539    2.360934
32540    0.581014
32541    0.295565
32542    2.435727
32543    0.581014
32544   -0.418055
32545    0.152841
32546    0.010117
32547    0.266744
32548    1.912179
32549    0.438289
32550    0.266744
32551   -0.555974
32552    0.266744
32553   -0.555974
32554    1.014669
32555   -1

#### Filter
Get a count of the number of observations in the 'age' series.

In [311]:
dfc['age'].value_counts()

36    898
31    888
34    886
23    877
35    876
33    875
28    867
30    861
37    858
25    841
27    835
32    828
38    827
39    816
29    813
41    808
24    798
40    794
26    785
42    780
43    770
22    765
20    753
46    737
45    734
44    724
21    720
19    712
47    708
50    602
     ... 
60    312
61    300
62    258
63    230
64    208
65    178
67    151
66    150
68    120
69    108
70     89
71     72
72     67
73     64
74     51
76     46
75     45
90     43
77     29
78     23
80     22
79     22
81     20
82     12
84     10
83      6
85      3
88      3
87      1
86      1
Name: age, Length: 73, dtype: int64

Group the "age" series and filter out ages with less then 602 observations.  Checking the previous step, 50 years is the value that corresponds to a count of 602 observations.  The resulting data frame, dfc_count_above_602, should have a maximum age value of 50 years.

In [312]:
dfc_count_above_602 = dfc.groupby('age').filter(lambda x: x['age'].count() >= 602)
dfc_count_above_602

,age,workclass,final weight,education,education num,marital status,occupation,relationship,race,sex,capital gain,capital loss,hours per week,native country,result
0,39,State gov,77516,Bachelors,13,Never married,Adm clerical,Not in family,White,Male,2174,0,40,United States,<=50K
1,50,Self emp not inc,83311,Bachelors,13,Married civ spouse,Exec managerial,Husband,White,Male,0,0,13,United States,<=50K
2,38,Private,215646,HS grad,9,Divorced,Handlers cleaners,Not in family,White,Male,0,0,40,United States,<=50K
4,28,Private,338409,Bachelors,13,Married civ spouse,Prof specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married civ spouse,Exec managerial,Wife,White,Female,0,0,40,United States,<=50K
8,31,Private,45781,Masters,14,Never married,Prof specialty,Not in family,White,Female,14084,0,50,United States,>50K
9,42,Private,159449,Bachelors,13,Married civ spouse,Exec managerial,Husband,White,Male,5178,0,40,United States,>50K
10,37,Private,280464,Some college,10,Married civ spouse,Exec managerial,Husband,Black,Male,0,0,80,United States,>50K
11,30,State gov,141297,Bachelors,13,Married civ spouse,Prof specialty,Husband,Asian Pac Islander,Male,0,0,40,India,>50K
12,23,Private,122272,Bachelors,13,Never married,Adm clerical,Own child,White,Female,0,0,30,United States,<=50K


Display the number of unique values in 'age'.  If the filter step above worked there should not be any ages over 50.

In [313]:
pd.value_counts(dfc_count_above_602['age'])

36    898
31    888
34    886
23    877
35    876
33    875
28    867
30    861
37    858
25    841
27    835
32    828
38    827
39    816
29    813
41    808
24    798
40    794
26    785
42    780
43    770
22    765
20    753
46    737
45    734
44    724
21    720
19    712
47    708
50    602
Name: age, dtype: int64

I'll find the max value on the 'age' series to make this check easier.  Again, the maximum age should be "50" if the filter step was correct.

In [314]:
dfc_count_above_602['age'].max()

50